In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from music21 import converter, note, stream
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Extract EMOPIA dataset
zip_path = "/content/drive/My Drive/EMOPIA_1.0.zip"
extract_path = "/content/EMOPIA"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset extracted to:", extract_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted to: /content/EMOPIA


In [ ]:
# Paths to dataset files
dataset_path = "/content/EMOPIA/EMOPIA_1.0"
midi_path = os.path.join(dataset_path, "midis")
label_csv_path = os.path.join(dataset_path, "label.csv")

# Load labels
labels_df = pd.read_csv(label_csv_path)
labels_df.columns = labels_df.columns.str.strip()  # Remove whitespace
labels_df['emotion'] = labels_df['4Q'].apply(lambda q: "happy" if q in [1, 4] else "sad")

# Filter only sad music
sad_df = labels_df[labels_df['emotion'] == 'sad']


In [ ]:
# Function to extract notes from a MIDI file
def extract_notes(file_path):
    midi_stream = converter.parse(file_path)
    notes = []
    for element in midi_stream.flatten().notes:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, note.Rest):
            notes.append('rest')
    return notes

# Extract notes from sad MIDI files
all_notes = []
for midi_file in os.listdir(midi_path):
    if midi_file.endswith('.mid'):
        file_id = midi_file.split('.')[0]
        if file_id in sad_df['ID'].values:
            file_path = os.path.join(midi_path, midi_file)
            notes = extract_notes(file_path)
            all_notes.append(notes)


In [ ]:
# Create mappings and prepare sequences
unique_notes = sorted(set(note for notes in all_notes for note in notes))
note_to_int = {note: i for i, note in enumerate(unique_notes)}
sequence_length = 100

# Convert notes to sequences
network_input = []
for notes in all_notes:
    for i in range(len(notes) - sequence_length):
        network_input.append([note_to_int[note] for note in notes[i:i + sequence_length]])

# Prepare data for LSTM
X = np.array(network_input)
X = np.reshape(X, (X.shape[0], sequence_length, 1)) / float(len(unique_notes))
y = np.array([note_to_int[notes[i + sequence_length]] for notes in all_notes for i in range(len(notes) - sequence_length)])
y = tf.keras.utils.to_categorical(y, num_classes=len(unique_notes))


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dropout(0.3),
    Dense(len(unique_notes), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100, 256)            │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 83)                  │          21,331 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 810,835 (3.09 MB)

 Trainable params: 810,835 (3.09 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

model_checkpoint = ModelCheckpoint(
    "/content/drive/My Drive/emopia_project/sad_model.keras",
    monitor="val_loss",
    save_best_only=True
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[model_checkpoint, early_stopping]
)


Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 811s 1s/step - loss: 4.0755 - val_loss: 3.9523
Epoch 2/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 870s 1s/step - loss: 3.9772 - val_loss: 3.9333
Epoch 3/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 868s 1s/step - loss: 3.9563 - val_loss: 3.9341
Epoch 4/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 856s 1s/step - loss: 3.9374 - val_loss: 3.9081
Epoch 5/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 827s 1s/step - loss: 3.9166 - val_loss: 3.9265
Epoch 6/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 781s 1s/step - loss: 3.9213 - val_loss: 3.9015
Epoch 7/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 782s 1s/step - loss: 3.8957 - val_loss: 3.8602
Epoch 8/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 800s 1s/step - loss: 3.8686 - val_loss: 3.7912
Epoch 9/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 806s 1s/step - loss: 3.7870 - val_loss: 3.7387
Epoch 10/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 768s 1s/step - loss: 3.7030 - val_loss: 3.6106


In [ ]:
import pickle

with open("/content/drive/My Drive/emopia_project/sad_metadata.pkl", "wb") as f:
    pickle.dump({"unique_notes": unique_notes, "note_to_int": note_to_int}, f)


In [ ]:
model.fit(
    X_train, y_train,
    epochs=10,  # Train for 10 more epochs
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[model_checkpoint, early_stopping]  # Reuse the same callbacks
)

print("Training for additional 10 epochs completed!")


Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 750s 1s/step - loss: 3.6054 - val_loss: 3.5693
Epoch 2/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 814s 1s/step - loss: 3.5202 - val_loss: 3.4703
Epoch 3/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 795s 1s/step - loss: 3.4425 - val_loss: 3.4052
Epoch 4/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 815s 1s/step - loss: 3.3573 - val_loss: 3.3322
Epoch 5/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 763s 1s/step - loss: 3.2800 - val_loss: 3.3000
Epoch 6/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 799s 1s/step - loss: 3.2065 - val_loss: 3.2388
Epoch 7/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 800s 1s/step - loss: 3.1400 - val_loss: 3.2089
Epoch 8/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 824s 1s/step - loss: 3.0688 - val_loss: 3.1723
Epoch 9/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 744s 1s/step - loss: 3.0168 - val_loss: 3.1481
Epoch 10/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 811s 1s/step - loss: 2.9559 - val_loss: 3.1293
Training for additional 10 epochs completed!


In [ ]:
from keras.models import load_model

model_path = "/content/drive/My Drive/emopia_project/sad_model.keras"
model = load_model(model_path)
print("Sad model loaded successfully!")


Sad model loaded successfully!


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks again
model_checkpoint = ModelCheckpoint(
    "/content/drive/My Drive/emopia_project/sad_model.keras",  # Same path
    monitor="val_loss",
    save_best_only=True
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,  # You can adjust this if needed
    restore_best_weights=True
)

# Continue training
history = model.fit(
    X_train, y_train,
    epochs=10,  # Additional 10 epochs
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[model_checkpoint, early_stopping]
)

print("Training for additional 10 epochs completed!")


Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 766s 1s/step - loss: 2.9570 - val_loss: 3.1042
Epoch 2/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 796s 1s/step - loss: 2.8836 - val_loss: 3.0836
Epoch 3/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 813s 1s/step - loss: 2.8134 - val_loss: 3.0859
Epoch 4/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 799s 1s/step - loss: 2.7726 - val_loss: 3.0804
Epoch 5/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 800s 1s/step - loss: 2.7418 - val_loss: 3.0544
Epoch 6/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 765s 1s/step - loss: 2.7050 - val_loss: 3.0498
Epoch 7/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 805s 1s/step - loss: 2.6843 - val_loss: 3.0579
Epoch 8/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 807s 1s/step - loss: 2.6364 - val_loss: 3.0435
Epoch 9/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 791s 1s/step - loss: 2.6050 - val_loss: 3.0312
Epoch 10/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 763s 1s/step - loss: 2.5923 - val_loss: 3.0400
Training for additional 10 epochs completed!


In [ ]:
import numpy as np

# Load the saved sad model
from keras.models import load_model
model = load_model("/content/drive/My Drive/emopia_project/sad_model.keras")
print("Sad model loaded!")

# Generate a seed sequence
seed_index = np.random.randint(0, len(X_val) - 1)  # Random seed from validation data
seed_sequence = X_val[seed_index]  # Shape: (sequence_length, input_dim)
print("Seed sequence selected!")

# Define temperature scaling function
def sample_with_temperature(predictions, temperature=1.0):
    predictions = np.log(predictions + 1e-9) / temperature  # Avoid log(0) errors
    exp_preds = np.exp(predictions)
    probabilities = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(probabilities), p=probabilities)

# Generate music with temperature scaling
generated_notes = []
current_sequence = seed_sequence
temperature = 1.2  # Adjust for diversity (higher = more randomness)

for _ in range(100):  # Generate 100 notes
    current_sequence = np.reshape(current_sequence, (1, current_sequence.shape[0], current_sequence.shape[1]))
    current_sequence = current_sequence / float(len(unique_notes))  # Normalize

    # Predict the next note probabilities
    predicted_probs = model.predict(current_sequence, verbose=0).flatten()
    predicted_note_int = sample_with_temperature(predicted_probs, temperature)

    # Append the predicted note
    generated_notes.append(predicted_note_int)

    # Update the sequence
    current_sequence = np.append(current_sequence[0, 1:], [[predicted_note_int]], axis=0)

# Convert generated notes back to note names
generated_notes_names = [unique_notes[note_int] for note_int in generated_notes]
print("Generated notes:", generated_notes_names)

# Convert the generated notes to a MIDI file
from music21 import stream, note

def create_midi_from_notes(notes, output_path):
    midi_stream = stream.Stream()
    for n in notes:
        if n == "rest":
            midi_stream.append(note.Rest())
        else:
            midi_stream.append(note.Note(n))
    midi_stream.write('midi', fp=output_path)

# Save the generated notes as a MIDI file
output_midi_path = "/content/drive/My Drive/emopia_project/generated_sad_music_with_diversity.mid"
create_midi_from_notes(generated_notes_names, output_midi_path)
print(f"Generated sad music with diversity saved to {output_midi_path}!")


Sad model loaded!
Seed sequence selected!
Generated notes: ['A4', 'B-4', 'A4', 'B-4', 'E5', 'G1', 'E5', 'B-4', 'A2', 'B-4', 'B4', 'A4', 'B3', 'A4', 'C#5', 'A4', 'D5', 'A3', 'D5', 'A3', 'A4', 'D2', 'A4', 'A4', 'A4', 'C#5', 'A4', 'A4', 'C#4', 'A4', 'B4', 'A4', 'C#4', 'B-4', 'B-4', 'A4', 'A4', 'B4', 'C#5', 'A4', 'F4', 'A4', 'C#6', 'C#5', 'F4', 'A2', 'E4', 'F#5', 'G1', 'B4', 'A4', 'A4', 'A4', 'A4', 'E4', 'A4', 'A4', 'C5', 'A4', 'A4', 'A4', 'B-4', 'B-4', 'B4', 'D5', 'A4', 'E4', 'B4', 'B4', 'A4', 'C#5', 'A2', 'B-5', 'A4', 'C5', 'F#4', 'A4', 'A4', 'A4', 'B-4', 'A4', 'D5', 'E-3', 'A4', 'A4', 'B-4', 'A5', 'C#5', 'B-4', 'E5', 'B-4', 'A4', 'A4', 'B-4', 'A4', 'B4', 'B1', 'B-4', 'B-4', 'A2']
Generated sad music with diversity saved to /content/drive/My Drive/emopia_project/generated_sad_music_with_diversity.mid!


In [ ]:
pip install pygame midi2audio


In [ ]:
from IPython.display import Audio

# Use the MIDI file to generate sound
!apt-get install -y timidity  # Install timidity if not already installed
!timidity {output_midi_path} -Ow -o output.wav
Audio("output.wav")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libao-common libao4
Suggested packages:
  fluid-soundfont-gs libaudio2 libsndio6.1 freepats pmidi timidity-daemon
The following NEW packages will be installed:
  fluid-soundfont-gm libao-common libao4 timidity
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 130 MB of archives.
After this operation, 151 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fluid-soundfont-gm all 3.1-5.3 [130 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao-common all 1.2.2+20180113-1.1ubuntu3 [6,568 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao4 amd64 1.2.2+20180113-1.1ubuntu3 [35.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 timidity amd64 2.14.0-8ubuntu1.22.04.1 [681 kB]
Fetched 130 MB in 6s (22.